# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 2 2022 1:43PM,238675,10,SO94277,"Senseonics, Incorporated",Released
1,Jun 2 2022 1:40PM,238674,19,ADV80003072,"AdvaGen Pharma, Ltd",Released
2,Jun 2 2022 1:40PM,238674,19,ADV80003073,"AdvaGen Pharma, Ltd",Released
3,Jun 2 2022 1:40PM,238674,19,ADV80003074,"AdvaGen Pharma, Ltd",Released
4,Jun 2 2022 1:40PM,238674,19,ADV80003075,"AdvaGen Pharma, Ltd",Released
5,Jun 2 2022 1:40PM,238674,19,ADV80003076,"AdvaGen Pharma, Ltd",Released
6,Jun 2 2022 1:40PM,238674,19,ADV80003077,"AdvaGen Pharma, Ltd",Released
7,Jun 2 2022 1:33PM,238673,20,SSNA-8096356,Sartorius Stedim North America,Released
8,Jun 2 2022 1:33PM,238672,10,8096359,Bio-PRF,Released
9,Jun 2 2022 1:29PM,238671,16,8096355,Sartorius Stedim Filters Inc.,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
47,238671,Released,1
48,238672,Released,1
49,238673,Released,1
50,238674,Released,6
51,238675,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
238671,NaN,NaN,1.0
238672,NaN,NaN,1.0
238673,NaN,NaN,1.0
238674,NaN,NaN,6.0
238675,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
238578,4.0,1.0,9.0
238597,0.0,21.0,1.0
238598,0.0,0.0,2.0
238613,1.0,2.0,0.0
238616,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
238578,4,1,9
238597,0,21,1
238598,0,0,2
238613,1,2,0
238616,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,238578,4,1,9
1,238597,0,21,1
2,238598,0,0,2
3,238613,1,2,0
4,238616,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,238578,4,1,9
1,238597,,21,1
2,238598,,,2
3,238613,1,2,
4,238616,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 2 2022 1:43PM,238675,10,"Senseonics, Incorporated"
1,Jun 2 2022 1:40PM,238674,19,"AdvaGen Pharma, Ltd"
7,Jun 2 2022 1:33PM,238673,20,Sartorius Stedim North America
8,Jun 2 2022 1:33PM,238672,10,Bio-PRF
9,Jun 2 2022 1:29PM,238671,16,Sartorius Stedim Filters Inc.
10,Jun 2 2022 1:22PM,238670,10,ISDIN Corporation
22,Jun 2 2022 1:13PM,238669,20,"ACI Healthcare USA, Inc."
33,Jun 2 2022 1:09PM,238668,10,Beach Products Inc
47,Jun 2 2022 1:09PM,238667,20,"ACI Healthcare USA, Inc."
56,Jun 2 2022 1:06PM,238666,10,"Methapharm, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 2 2022 1:43PM,238675,10,"Senseonics, Incorporated",,,1
1,Jun 2 2022 1:40PM,238674,19,"AdvaGen Pharma, Ltd",,,6
2,Jun 2 2022 1:33PM,238673,20,Sartorius Stedim North America,,,1
3,Jun 2 2022 1:33PM,238672,10,Bio-PRF,,,1
4,Jun 2 2022 1:29PM,238671,16,Sartorius Stedim Filters Inc.,,,1
5,Jun 2 2022 1:22PM,238670,10,ISDIN Corporation,,,12
6,Jun 2 2022 1:13PM,238669,20,"ACI Healthcare USA, Inc.",,,11
7,Jun 2 2022 1:09PM,238668,10,Beach Products Inc,,,14
8,Jun 2 2022 1:09PM,238667,20,"ACI Healthcare USA, Inc.",,,9
9,Jun 2 2022 1:06PM,238666,10,"Methapharm, Inc.",,5,3


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 2 2022 1:43PM,238675,10,"Senseonics, Incorporated",1,,
1,Jun 2 2022 1:40PM,238674,19,"AdvaGen Pharma, Ltd",6,,
2,Jun 2 2022 1:33PM,238673,20,Sartorius Stedim North America,1,,
3,Jun 2 2022 1:33PM,238672,10,Bio-PRF,1,,
4,Jun 2 2022 1:29PM,238671,16,Sartorius Stedim Filters Inc.,1,,
5,Jun 2 2022 1:22PM,238670,10,ISDIN Corporation,12,,
6,Jun 2 2022 1:13PM,238669,20,"ACI Healthcare USA, Inc.",11,,
7,Jun 2 2022 1:09PM,238668,10,Beach Products Inc,14,,
8,Jun 2 2022 1:09PM,238667,20,"ACI Healthcare USA, Inc.",9,,
9,Jun 2 2022 1:06PM,238666,10,"Methapharm, Inc.",3,5,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 2 2022 1:43PM,238675,10,"Senseonics, Incorporated",1,,
1,Jun 2 2022 1:40PM,238674,19,"AdvaGen Pharma, Ltd",6,,
2,Jun 2 2022 1:33PM,238673,20,Sartorius Stedim North America,1,,
3,Jun 2 2022 1:33PM,238672,10,Bio-PRF,1,,
4,Jun 2 2022 1:29PM,238671,16,Sartorius Stedim Filters Inc.,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 2 2022 1:43PM,238675,10,"Senseonics, Incorporated",1.0,NaN,NaN
1,Jun 2 2022 1:40PM,238674,19,"AdvaGen Pharma, Ltd",6.0,NaN,NaN
2,Jun 2 2022 1:33PM,238673,20,Sartorius Stedim North America,1.0,NaN,NaN
3,Jun 2 2022 1:33PM,238672,10,Bio-PRF,1.0,NaN,NaN
4,Jun 2 2022 1:29PM,238671,16,Sartorius Stedim Filters Inc.,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 2 2022 1:43PM,238675,10,"Senseonics, Incorporated",1.0,0.0,0.0
1,Jun 2 2022 1:40PM,238674,19,"AdvaGen Pharma, Ltd",6.0,0.0,0.0
2,Jun 2 2022 1:33PM,238673,20,Sartorius Stedim North America,1.0,0.0,0.0
3,Jun 2 2022 1:33PM,238672,10,Bio-PRF,1.0,0.0,0.0
4,Jun 2 2022 1:29PM,238671,16,Sartorius Stedim Filters Inc.,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3579834,96.0,29.0,4.0
12,954561,8.0,4.0,1.0
15,1431714,13.0,36.0,20.0
16,954608,3.0,1.0,0.0
19,954626,15.0,4.0,0.0
20,1431906,34.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3579834,96.0,29.0,4.0
1,12,954561,8.0,4.0,1.0
2,15,1431714,13.0,36.0,20.0
3,16,954608,3.0,1.0,0.0
4,19,954626,15.0,4.0,0.0
5,20,1431906,34.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,96.0,29.0,4.0
1,12,8.0,4.0,1.0
2,15,13.0,36.0,20.0
3,16,3.0,1.0,0.0
4,19,15.0,4.0,0.0
5,20,34.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,96.0
1,12,Released,8.0
2,15,Released,13.0
3,16,Released,3.0
4,19,Released,15.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20
Status,,,,,,
Completed,4.0,1.0,20.0,0.0,0.0,0.0
Executing,29.0,4.0,36.0,1.0,4.0,0.0
Released,96.0,8.0,13.0,3.0,15.0,34.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20
0,Completed,4.0,1.0,20.0,0.0,0.0,0.0
1,Executing,29.0,4.0,36.0,1.0,4.0,0.0
2,Released,96.0,8.0,13.0,3.0,15.0,34.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20
0,Completed,4.0,1.0,20.0,0.0,0.0,0.0
1,Executing,29.0,4.0,36.0,1.0,4.0,0.0
2,Released,96.0,8.0,13.0,3.0,15.0,34.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()